In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [2]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [3]:
path = '/home/'

In [4]:
train_png = sorted(glob(path + 'open/train/*.png'))
test_png = sorted(glob(path + 'open/test/*.png'))

In [5]:
len(train_png), len(test_png)

(4277, 2154)

In [6]:
train_y = pd.read_csv(path +"open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [7]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [8]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

100%|██████████| 2154/2154 [01:26<00:00, 25.02it/s]


In [9]:
np.save(path + 'train_imgs_384', np.array(train_imgs))
np.save(path + 'test_imgs_384', np.array(test_imgs))

In [10]:
train_imgs = np.load(path + 'train_imgs_384.npy')
test_imgs = np.load(path + 'test_imgs_384.npy')

In [11]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488


In [12]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663


In [13]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('densenet169', pretrained=True, num_classes=88)
        if self.mode == 'test':
          self.model = timm.create_model('densenet169', pretrained=True, num_classes=88)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [14]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [15]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [16]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 16
epochs = 100
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
  print("----------fold_{} start!----------".format(idx))
  t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
  t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

  # Train
  train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
  train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

  # Val
  val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
  val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

  gc.collect()
  torch.cuda.empty_cache()
  best=0

  model = Network().to(device)

  optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay = 1e-3)
  criterion = nn.CrossEntropyLoss()
  scaler = torch.cuda.amp.GradScaler()  

  best_f1 = 0
  early_stopping = 0
  for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
    train_f1 = score_function(train_y, train_pred)
    state_dict= model.state_dict()
    model.eval()
    with torch.no_grad():
      val_loss = 0 
      val_pred = []
      val_y = []
      

      for batch in (val_loader):
        x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
        y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred_val = model(x_val)
        loss_val = criterion(pred_val, y_val)

        val_loss += loss_val.item()/len(val_loader)
        val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
        val_y += y_val.detach().cpu().numpy().tolist()
      val_f1 = score_function(val_y, val_pred)

      if val_f1 > best_f1:
        best_epoch = epoch
        best_loss = val_loss
        best_f1 = val_f1
        early_stopping = 0

        torch.save({'epoch':epoch,
                    'state_dict':state_dict,
                    'optimizer': optimizer.state_dict(),
                    'scaler': scaler.state_dict(),
             }, path +'best_model_{}.pth'.format(idx))
        print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
      else:
          early_stopping += 1

            # Early Stopping
      if early_stopping == 20:
        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
        break

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
    print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------


Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to /root/.cache/torch/hub/checkpoints/densenet169-b2777c0a.pth


-----------------SAVE:1 epoch----------------
epoch : 1/100    time : 75s/7426s
TRAIN    loss : 1.16104    f1 : 0.15331
Val    loss : 0.72507    f1 : 0.18888
-----------------SAVE:2 epoch----------------
epoch : 2/100    time : 70s/6908s
TRAIN    loss : 0.65932    f1 : 0.21630
Val    loss : 0.50614    f1 : 0.24761
-----------------SAVE:3 epoch----------------
epoch : 3/100    time : 73s/7083s
TRAIN    loss : 0.49948    f1 : 0.29449
Val    loss : 0.44304    f1 : 0.32044
-----------------SAVE:4 epoch----------------
epoch : 4/100    time : 70s/6753s
TRAIN    loss : 0.43251    f1 : 0.38576
Val    loss : 0.51275    f1 : 0.36518
-----------------SAVE:5 epoch----------------
epoch : 5/100    time : 69s/6554s
TRAIN    loss : 0.36727    f1 : 0.46340
Val    loss : 0.32421    f1 : 0.52729
-----------------SAVE:6 epoch----------------
epoch : 6/100    time : 68s/6409s
TRAIN    loss : 0.29649    f1 : 0.55715
Val    loss : 0.24254    f1 : 0.58657
-----------------SAVE:7 epoch----------------
epoch 

-----------------SAVE:26 epoch----------------
epoch : 26/100    time : 73s/5372s
TRAIN    loss : 0.01970    f1 : 0.97772
Val    loss : 0.13455    f1 : 0.82897
epoch : 27/100    time : 70s/5138s
TRAIN    loss : 0.04344    f1 : 0.96832
Val    loss : 0.23859    f1 : 0.75107
epoch : 28/100    time : 69s/4949s
TRAIN    loss : 0.04771    f1 : 0.94310
Val    loss : 0.20809    f1 : 0.75261
epoch : 29/100    time : 71s/5062s
TRAIN    loss : 0.03820    f1 : 0.96113
Val    loss : 0.19170    f1 : 0.79338
epoch : 30/100    time : 70s/4874s
TRAIN    loss : 0.05775    f1 : 0.93577
Val    loss : 0.33752    f1 : 0.72614
epoch : 31/100    time : 72s/4998s
TRAIN    loss : 0.05411    f1 : 0.92900
Val    loss : 0.16974    f1 : 0.76014
epoch : 32/100    time : 68s/4625s
TRAIN    loss : 0.03319    f1 : 0.96255
Val    loss : 0.35787    f1 : 0.75274
epoch : 33/100    time : 69s/4611s
TRAIN    loss : 0.04190    f1 : 0.96090
Val    loss : 0.17792    f1 : 0.80731
epoch : 34/100    time : 71s/4692s
TRAIN    loss 

epoch : 33/100    time : 68s/4580s
TRAIN    loss : 0.04514    f1 : 0.95266
Val    loss : 0.18316    f1 : 0.76871
epoch : 34/100    time : 71s/4709s
TRAIN    loss : 0.03429    f1 : 0.96509
Val    loss : 0.16442    f1 : 0.76777
epoch : 35/100    time : 68s/4439s
TRAIN    loss : 0.03026    f1 : 0.97261
Val    loss : 0.15393    f1 : 0.81584
epoch : 36/100    time : 69s/4432s
TRAIN    loss : 0.05035    f1 : 0.94786
Val    loss : 0.19187    f1 : 0.78920
epoch : 37/100    time : 71s/4491s
TRAIN    loss : 0.02772    f1 : 0.97043
Val    loss : 0.19563    f1 : 0.76044
epoch : 38/100    time : 69s/4295s
TRAIN    loss : 0.04322    f1 : 0.95700
Val    loss : 0.19861    f1 : 0.75087
epoch : 39/100    time : 70s/4247s
TRAIN    loss : 0.06038    f1 : 0.93711
Val    loss : 0.18050    f1 : 0.79927
epoch : 40/100    time : 71s/4232s
TRAIN    loss : 0.01850    f1 : 0.98764
Val    loss : 0.16769    f1 : 0.78469
epoch : 41/100    time : 68s/4012s
TRAIN    loss : 0.02706    f1 : 0.97412
Val    loss : 0.14496

-----------------SAVE:7 epoch----------------
epoch : 7/100    time : 72s/6659s
TRAIN    loss : 0.22914    f1 : 0.65959
Val    loss : 0.23400    f1 : 0.60990
-----------------SAVE:8 epoch----------------
epoch : 8/100    time : 70s/6418s
TRAIN    loss : 0.18860    f1 : 0.73469
Val    loss : 0.24111    f1 : 0.68701
epoch : 9/100    time : 70s/6413s
TRAIN    loss : 0.17390    f1 : 0.73986
Val    loss : 0.19399    f1 : 0.63250
-----------------SAVE:10 epoch----------------
epoch : 10/100    time : 72s/6437s
TRAIN    loss : 0.14077    f1 : 0.79045
Val    loss : 0.22849    f1 : 0.71354
epoch : 11/100    time : 71s/6318s
TRAIN    loss : 0.13380    f1 : 0.83493
Val    loss : 0.25458    f1 : 0.65347
epoch : 12/100    time : 70s/6137s
TRAIN    loss : 0.11937    f1 : 0.83978
Val    loss : 0.17342    f1 : 0.70465
-----------------SAVE:13 epoch----------------
epoch : 13/100    time : 70s/6117s
TRAIN    loss : 0.09362    f1 : 0.87757
Val    loss : 0.18081    f1 : 0.73172
epoch : 14/100    time : 7

In [17]:
pred_ensemble = []
batch_size = 16
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load((path+'best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [18]:
len(pred_ensemble)

5

In [19]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [20]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [21]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [22]:
submission.to_csv(path + "densenet169_norm_epoch100.csv", index = False)

In [23]:
# d2b19ece8c7374053ee1fd80cfe419ddfc640c01f9ebe4cbd5caeb9f1906974a

from dacon_submit_api import dacon_submit_api 

result = dacon_submit_api.post_submission_file(
'/home/densenet169_norm_epoch100.csv', 
'd2b19ece8c7374053ee1fd80cfe419ddfc640c01f9ebe4cbd5caeb9f1906974a', 
'235894', 
'mean', 
'densenet169_BS16' )

# https://www.dacon.io/competitions/official/235894/overview/rules

{'isSubmitted': False, 'detail': 'Over max submission count of Daily. 일일 제출 가능한 최대 횟수가 초과 되었습니다.'}


### 모델 학습

사전 학습 모델의 성능 파악을 할 때 Fold 학습은 실행 시간이 오래걸려서 fold를 나누지 않은 데이터에 대해서 학습을 진행하고 성능을 비교하였습니다.

In [18]:
batch_size = 16
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [19]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 16
epochs = 30

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

### 추론

In [80]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

### 제출물 생성

In [ ]:
submission = pd.read_csv(path + "open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv(path + "VGG16_norm.csv", index = False)